# Spotify RecSys Engine | Data Mining - Final Project

## Section 5: Playlist creation for beta testing



In [63]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import requests
import json
import random
from random import choice, randrange
import urllib
import urllib.parse
from pprint import pprint
import webbrowser
import base64
from collections import MutableMapping 
import string
import os
import time
from math import floor
%load_ext jupyternotify


The jupyternotify extension is already loaded. To reload it, use:
  %reload_ext jupyternotify


### Global variables

We first read in a file that contains the client API keys, as define various global variables. 

In [70]:
# .py file that includes CLIENT_ID and CLIENT_SECRET keys
from client_info import * 

# call-back URL for OAuth 2.0
redirect_uri = "https://example.com/callback" 

# list of Spotify account scopes/user permissions to request
scopes_list = ["ugc-image-upload", 
    "user-read-playback-state", 
    "user-modify-playback-state", 
    "user-read-currently-playing", 
    "streaming", 
    "app-remote-control", 
    "user-read-email",                
    "user-read-private", 
    "playlist-read-collaborative", 
    "playlist-modify-public", 
    "playlist-read-private", 
    "playlist-modify-private", 
    "user-library-modify", 
    "user-library-read", 
    "user-top-read", 
    "user-read-recently-played", 
    "user-follow-read", 
    "user-follow-modify"
] 
scope_string = '%20'.join(scopes_list)

# user_data_dict endpoint:"https://api.spotify.com/v1/me/"

# exhaustive list of categories of data available regarding a Spotify user
user_data_dict = {
    "profile":""
}


### Section 5.1: HTTP requests and OAuth 2.0 protocol

We use the `requests` module to perform `GET` and `POST` requests. Whilst there exists a SpotiPy package out there, building our own implementation allows for a more specific and flexible program for our application. 

We initialise by requesting the user's authentication through the Spotify authorisation endpoint. The following function would open a new tab and asks the user to log in and grant the list of permissions detailed in `scopes_list`. Then the user will be re-directed to a callback URL which contains a `code` and a `state`,  with which we then use to request `refresh` and `access` tokens. Access tokens time out after 3600 s, and so the refresh token is used to periodically request new access tokens. 


In [65]:
def user_auth():
    # request user authorization and request refresh and access tokens
    options_dict = {"client_id":CLIENT_ID,
        "response_type":"code",
        "redirect_uri":urllib.parse.quote_plus(redirect_uri),
        "state":str(random.getrandbits(128)),
        "scope":scope_string,
        "show_dialog":"true"
        }
    endpoint = "https://accounts.spotify.com/authorize"
    r = requests.get(endpoint + "?" + "&".join([key + "=" + value for key, value in options_dict.items()]), allow_redirects=True)
    webbrowser.open(r.url) 
    callback_url = input("Enter the callback URL provided upon authentication: ")
    code = callback_url.strip("https://example.com/callback?code=").split("&state=")[0]
    state = callback_url.strip("https://example.com/callback?code=").split("&state=")[1]
    auth_str = '{}:{}'.format(CLIENT_ID, CLIENT_SECRET)
    b64_auth_str = base64.b64encode(auth_str.encode()).decode()
    header = {'Authorization':'Basic {}'.format(b64_auth_str)}
    data = {
        'grant_type':'authorization_code',
        'code':code,
        'redirect_uri':redirect_uri
        }
    auth = requests.post('https://accounts.spotify.com/api/token', headers=header, data=data)
    global auth_json
    auth_json = json.loads(auth.text)


The following function requests new access tokens with the refresh token. 

In [66]:
def get_token(auth_json):
    # request new tokens using refresh_token
    client_auth_str = '{}:{}'.format(CLIENT_ID, CLIENT_SECRET)
    b64_client_auth_str = base64.b64encode(client_auth_str.encode()).decode()
    header = {'Authorization':'Basic {}'.format(b64_client_auth_str)}
    data = {"grant_type":"refresh_token", "refresh_token":auth_json["refresh_token"]}
    global refresh
    refresh = requests.post('https://accounts.spotify.com/api/token', headers=header, data=data)
    refresh_json = json.loads(refresh.text)
    global refreshed_token
    refreshed_token = refresh_json["access_token"]


### Section 5.2: Acquiring Spotify ID

The user's Spptify ID is required in the creation of a playlist through the API. The ID is stored within the user profile.

In [104]:
def get_user_data(user_element):
    # request an aspect of user data
    get_token(auth_json)
    headers = {
        'Accept':'application/json',
        'Content-Type':'application/json',
        'Authorization':'Bearer {}'.format(refreshed_token)
        }
    endpoint = "https://api.spotify.com/v1/me/"
    url = endpoint + user_data_dict[user_element]
    user_info = requests.get(url=url, headers=headers)
    user_info_dict = json.loads(user_info.text)
    if "next" in user_info_dict and user_info_dict["next"] is not None:
        more_user_info_url = user_info_dict["next"] 
        while  more_user_info_url is not None:
            get_token(auth_json)
            # grab more user data if total > limit=50
            headers = {
                'Accept':'application/json',
                'Content-Type':'application/json',
                'Authorization':'Bearer {}'.format(refreshed_token)
                }
            more_user_info = requests.get(url=more_user_info_url, headers=headers) #
            more_user_info_dict = json.loads(more_user_info.text) # 
            more_user_info_url = more_user_info_dict["next"] #
            user_info_dict["items"].extend(more_user_info_dict["items"]) 
    return user_info_dict

def get_master_user_profile():
    # request all user data and assemble dict
    global master_user_profile
    master_user_profile = {key:get_user_data(key) for key in user_data_dict}


## Section 5.3: Playlist creation

With the various sample playlists created using ML methods, we now performed a blind test to see if the user likes the playlists. The playlists are loaded in as `.csv` files, with a column populated by Spotify track URI's. We assign random numbers to each of the playlists and ask if the user can identify if it belongs to one of the three categories (1) Chill, (2) Dance, (3) Discover. 

The user should listen to the tracks within each playlist with Shuffle mode on. 

Store all playlists in a sub-directory `./Test_Playlists/`. This code handles playlists up to 100 tracks in length. 

In [108]:
from os import listdir
from os.path import isfile, join
import csv

def create_test_playlists():
    user_auth()
    mypath = "./Test_Playlists/"
    playlists_csvs = [mypath + f for f in listdir(mypath) if isfile(join(mypath, f))]
    playlist_ids = list(range(len(playlists_csvs)))
    playlist_ids = ["Test_Playlist_" + str(id) for id in playlist_ids]
    random.shuffle(playlist_ids)
    encoded_playlists = list(zip(playlist_ids, playlists_csvs))
    with open("encoded_playlists.csv","w+") as f:
        csvWriter = csv.writer(f,delimiter=',')
        csvWriter.writerows(encoded_playlists)

    for encoded_playlist in encoded_playlists: 
        encoded = encoded_playlist[0]
        path = encoded_playlist[1]
        playlist_pd = pd.read_csv(path)[["name", "artist_name","uri"]]
        playlist_pd.to_csv(mypath + encoded + ".csv", index=False)
        track_uris = playlist_pd["uri"].values.tolist()
        get_token(auth_json)
        headers = {
            'Accept':'application/json',
            'Content-Type':'application/json',
            'Authorization':'Bearer {}'.format(refreshed_token)
            }
        create_playlist_endpoint = "https://api.spotify.com/v1/users/{}/playlists".format(user_id)
        create_playlist_param = {
            "name":encoded,
            "description":"BA@UChicagoMSCA"
        }
        create_playlist = requests.post(create_playlist_endpoint, headers=headers, data=json.dumps(create_playlist_param))
        create_playlist_dict = json.loads(create_playlist.text)
        created_playlist_uri = create_playlist_dict["uri"].split(":")[2]  

        get_token(auth_json)
        headers = {
            'Accept':'application/json',
            'Content-Type':'application/json',
            'Authorization':'Bearer {}'.format(refreshed_token)
            }
        add_tracks_to_playlist_endpoint = "https://api.spotify.com/v1/playlists/{}/tracks".format(created_playlist_uri)
        add_tracks_param = {
                "uris":track_uris
        }
        add_tracks = requests.post(add_tracks_to_playlist_endpoint, headers=headers, data=json.dumps(add_tracks_param))
        add_tracks_dict = json.loads(add_tracks.text)


In [109]:
create_test_playlists()